# Naive Bayes classifiers

Our goal is to "train" a NB classifier (really just count things) and apply it to some new documents.

In [1]:
import numpy as np
from collections import Counter
import regex

Simple word definition, we could do more complicated things like identify latex segments.

In [2]:
word_pattern = regex.compile("\w+")

In [3]:
def read_file(filename):
    docs = []
    sum_counter = Counter()
    total_tokens = 0
    
    with open(filename) as reader:
        for line in reader:
            fields = line.rstrip().split("\t")
            if len(fields) < 3:
                continue
                
            tokens = word_pattern.findall(fields[2])
            counts = Counter(tokens)
            sum_counter.update(tokens)
            total_tokens += len(tokens)
            
            doc = { "name": fields[0], "label": fields[1], "tokens": tokens, "counts": counts }
            docs.append(doc)
    
    return (sum_counter, total_tokens, docs)

In [4]:
ds_counter, ds_tokens, ds_docs = read_file("datascience10k.txt")
stat_counter, stat_tokens, stat_docs = read_file("stats10k.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'datascience10k.txt'

In [5]:
print(ds_tokens, stat_tokens)

1260636 1636418


In [6]:
ds_counter.most_common(10)

[('the', 60439),
 ('to', 34888),
 ('a', 28704),
 ('of', 27683),
 ('I', 25196),
 ('is', 24455),
 ('and', 22722),
 ('in', 16448),
 ('you', 14814),
 ('that', 14015)]

In [7]:
stat_counter.most_common(10)

[('the', 84321),
 ('of', 39721),
 ('to', 37171),
 ('a', 35188),
 ('is', 33199),
 ('I', 28313),
 ('and', 28147),
 ('in', 20997),
 ('that', 19814),
 ('for', 16206)]

In [8]:
# create a union vocabulary so we have one list of all the words we know about

everything_counter = ds_counter + stat_counter
vocabulary = list(everything_counter.keys())
vocab_size = len(vocabulary)
vocab_set = set(vocabulary)

In [9]:
vocab_size

50059

## Zipf's law

Almost half the distinct terms occur exactly once, but about 100 words make up half of all the tokens.

In [10]:
# there are 23k word types that occur once 

Counter(everything_counter.values()).most_common(10)

[(1, 23095),
 (2, 6626),
 (3, 3387),
 (4, 2264),
 (5, 1565),
 (6, 1174),
 (7, 879),
 (8, 748),
 (9, 631),
 (10, 499)]

In [11]:
# 10 words make up about 20% of the collection

cumulative_proportion = 0.0

for w, c in everything_counter.most_common(100):
    proportion = c / (ds_tokens + stat_tokens)
    cumulative_proportion += proportion
    
    print("{}\t{:.4f}\t{:.1f}%\t{}".format(c, proportion, 100 * cumulative_proportion, w))

144760	0.0500	5.0%	the
72059	0.0249	7.5%	to
67404	0.0233	9.8%	of
63892	0.0221	12.0%	a
57654	0.0199	14.0%	is
53509	0.0185	15.9%	I
50869	0.0176	17.6%	and
37445	0.0129	18.9%	in
33829	0.0117	20.1%	that
29222	0.0101	21.1%	for
29149	0.0101	22.1%	you
22169	0.0077	22.8%	it
21416	0.0074	23.6%	this
21251	0.0073	24.3%	1
21169	0.0073	25.1%	are
21055	0.0073	25.8%	be
19968	0.0069	26.5%	with
18786	0.0065	27.1%	have
18145	0.0063	27.7%	data
16914	0.0058	28.3%	on
16789	0.0058	28.9%	as
16514	0.0057	29.5%	can
15049	0.0052	30.0%	not
13553	0.0047	30.5%	model
13192	0.0046	30.9%	2
12758	0.0044	31.4%	0
12640	0.0044	31.8%	or
11936	0.0041	32.2%	The
11788	0.0041	32.6%	t
11216	0.0039	33.0%	your
10943	0.0038	33.4%	but
10510	0.0036	33.7%	from
10504	0.0036	34.1%	if
10466	0.0036	34.5%	s
9936	0.0034	34.8%	which
9782	0.0034	35.1%	an
9688	0.0033	35.5%	would
9272	0.0032	35.8%	by
8733	0.0030	36.1%	use
8493	0.0029	36.4%	one
8120	0.0028	36.7%	we
8056	0.0028	37.0%	will
7898	0.0027	37.2%	x
7648	0.0026	37.5%	there
7521	0.0026	3

In [12]:
# turn counts into probability with Dirichlet smoothing

def log_prob(word, counts, total, smoothing=0.1):
    prob = (counts[word] + smoothing) / (total + vocab_size * smoothing)
    return np.log( prob ) 

In [13]:
# two example docs I'm pretty sure aren't in the training set

test_stat_doc = "Suppose X1 and X2 are independent. Both have Bernoulli distribution, equal to one with probability 0.5. What is the probability distribution of the product X1X2? What is the covariance between X1 and X1X2? I am a beginner at statistics and quite confused by these conceptions. Would you please offer some guidance? Thank you!"
test_ds_doc = "Can you tell me cheap services to train ai models in the cloud, and that they are easy to configure. I would like something like Google Collab, simple but with more powerful gpu. I want the service to come with python installed so that I can program in the cloud directly and not install python on my pc."

In [14]:
def compare_tokens(doc, smoothing=0.1):
    total = 0
    for token in word_pattern.findall(doc):
        if token in vocab_set:
            ds_log_prob = log_prob(token, ds_counter, ds_tokens, smoothing)
            stat_log_prob = log_prob(token, stat_counter, stat_tokens, smoothing)
            diff = ds_log_prob - stat_log_prob
            total += diff
            print(f"{ds_log_prob:.02f}\t{stat_log_prob:.02f}\t{diff:.02f}\t{token}")
        else:
            print("skipping ", token)
    print(total)

## Intuition for log probabilities

It's helpful to have a rough idea what the scale of a log probability means. All log probabilities are negative or zero (why?) so if we drop the negative and exponentiate, we get the equivalent "one in ..." number.

Note that in this class $\log$ means natural log.

In [15]:
np.log(1/10)

-2.3025850929940455

In [16]:
np.exp(2.302586)

10.000009070063655

In [17]:
for lp in range(1,15):
    print("log prob -{} ~ one in {: 10.1f}".format(lp, np.exp(lp)))

log prob -1 ~ one in        2.7
log prob -2 ~ one in        7.4
log prob -3 ~ one in       20.1
log prob -4 ~ one in       54.6
log prob -5 ~ one in      148.4
log prob -6 ~ one in      403.4
log prob -7 ~ one in     1096.6
log prob -8 ~ one in     2981.0
log prob -9 ~ one in     8103.1
log prob -10 ~ one in    22026.5
log prob -11 ~ one in    59874.1
log prob -12 ~ one in   162754.8
log prob -13 ~ one in   442413.4
log prob -14 ~ one in  1202604.3


In [18]:
np.exp(-1.4)

0.2465969639416065

In [19]:
ds_counter["Python"]

640

In [20]:
stat_counter["Python"]

134

In [21]:
# Capitalization is consistently more stats-ey (negative)
# (including "the" as a baseline)

compare_tokens("R r Python python The the")

-7.22	-6.47	-0.75	R
-8.75	-7.77	-0.98	r
-7.59	-9.41	1.82	Python
-7.85	-9.98	2.12	python
-5.56	-5.45	-0.11	The
-3.04	-2.97	-0.07	the
2.030811072432871


In [22]:
compare_tokens(test_stat_doc, 10.0)

-9.31	-8.55	-0.75	Suppose
-11.05	-10.01	-1.04	X1
-4.35	-4.33	-0.02	and
-11.34	-10.21	-1.13	X2
-5.25	-5.19	-0.06	are
-8.84	-7.41	-1.44	independent
-9.79	-9.77	-0.02	Both
-5.29	-5.38	0.09	have
-11.12	-9.89	-1.23	Bernoulli
-7.64	-6.25	-1.39	distribution
-8.81	-7.98	-0.83	equal
-3.92	-4.05	0.13	to
-6.12	-6.13	0.00	one
-5.25	-5.30	0.05	with
-7.81	-6.80	-1.01	probability
-5.93	-5.58	-0.35	0
-7.29	-6.95	-0.33	5
-7.19	-7.26	0.08	What
-4.28	-4.16	-0.11	is
-3.37	-3.23	-0.14	the
-7.81	-6.80	-1.01	probability
-7.64	-6.25	-1.39	distribution
-4.15	-3.99	-0.17	of
-3.37	-3.23	-0.14	the
-8.62	-8.73	0.11	product
-11.98	-12.27	0.29	X1X2
-7.19	-7.26	0.08	What
-4.28	-4.16	-0.11	is
-3.37	-3.23	-0.14	the
-10.08	-8.36	-1.72	covariance
-6.92	-6.63	-0.30	between
-11.05	-10.01	-1.04	X1
-4.35	-4.33	-0.02	and
-11.98	-12.27	0.29	X1X2
-4.25	-4.32	0.08	I
-6.38	-6.52	0.15	am
-4.12	-4.11	-0.01	a
-10.09	-11.11	1.02	beginner
-6.43	-6.23	-0.20	at
-8.91	-8.03	-0.89	statistics
-4.35	-4.33	-0.02	and
-8.42	-8.32	-0.10	quite
-

In [23]:
np.exp(7.439)

1701.0483220209924

In [24]:
compare_tokens(test_ds_doc, 10)

-8.21	-8.33	0.12	Can
-4.78	-5.00	0.23	you
-8.92	-8.82	-0.10	tell
-7.11	-7.17	0.06	me
-11.20	-11.48	0.28	cheap
-11.01	-11.21	0.19	services
-3.92	-4.05	0.13	to
-7.00	-8.77	1.77	train
-10.51	-12.18	1.67	ai
-7.14	-7.18	0.04	models
-4.67	-4.62	-0.05	in
-3.37	-3.23	-0.14	the
-10.13	-11.40	1.26	cloud
-4.35	-4.33	-0.02	and
-4.83	-4.68	-0.15	that
-6.89	-6.85	-0.04	they
-5.25	-5.19	-0.06	are
-8.65	-8.83	0.18	easy
-3.92	-4.05	0.13	to
-11.49	-11.87	0.38	configure
-4.25	-4.32	0.08	I
-6.04	-5.96	-0.07	would
-6.18	-6.52	0.34	like
-7.55	-7.66	0.11	something
-6.18	-6.52	0.34	like
-9.06	-10.42	1.35	Google
-11.98	-12.27	0.29	Collab
-7.72	-7.77	0.05	simple
-5.86	-5.88	0.02	but
-5.25	-5.30	0.05	with
-6.33	-6.46	0.13	more
-10.01	-10.37	0.36	powerful
-11.09	-12.27	1.19	gpu
-4.25	-4.32	0.08	I
-6.55	-6.75	0.20	want
-3.37	-3.23	-0.14	the
-10.62	-10.86	0.24	service
-3.92	-4.05	0.13	to
-8.77	-8.78	0.01	come
-5.25	-5.30	0.05	with
-8.16	-10.12	1.96	python
-10.43	-11.48	1.05	installed
-6.40	-6.39	-0.02	so
-4.83	-4.6

## Evaluation as a classifier

NB isn't the best classifier, but it's pretty close, and the fact that it's just counting things makes leave-one-out cross validation really easy. We don't want to test on the training set, but we can easily create a test set of one for each doc by pretending we haven't seen it.

Note that I'm leaving out the prior probability of data science vs. stats, since I made this 50/50 by construction.

In [25]:
ds_tokens

1260636

In [26]:
smoothing = 1.0

num_correct = 0
num_docs = 0

for doc in ds_docs:
    # pretend we haven't seen this before
    for word, count in doc["counts"].items():
        ds_counter[word] -= count
        ds_tokens -= count
    
    total_diff = 0.0
    for token in doc["tokens"]:
        if token in vocab_set:
            ds_log_prob = log_prob(token, ds_counter, ds_tokens, smoothing)
            stat_log_prob = log_prob(token, stat_counter, stat_tokens, smoothing)
            diff = ds_log_prob - stat_log_prob
            total_diff += diff
    
    if total_diff > 0.0:
        num_correct += 1
    num_docs += 1
    
    # add it back in
    for word, count in doc["counts"].items():
        ds_counter[word] += count
        ds_tokens += count
    
    if ds_tokens != 1260636:
        print(doc)
        break

print(f"DS\t{num_correct}\t{num_docs}")

DS	8882	9913


In [27]:
ds_tokens

1260636

In [28]:
smoothing = 1.0

num_correct = 0
num_docs = 0

for doc in stat_docs:
    # pretend we haven't seen this before
    for word, count in doc["counts"].items():
        stat_counter[word] -= count
        stat_tokens -= count
    
    total = 0.0
    for token in doc["tokens"]:
        if token in vocab_set:
            ds_log_prob = log_prob(token, ds_counter, ds_tokens, smoothing)
            stat_log_prob = log_prob(token, stat_counter, stat_tokens, smoothing)
            diff = ds_log_prob - stat_log_prob
            total += diff
    
    if total < 0.0:
        num_correct += 1
    num_docs += 1
    
    # add it back in
    for word, count in doc["counts"].items():
        stat_counter[word] += count
        stat_tokens += count

print(f"stat\t{num_correct}\t{num_docs}")

stat	7421	9937
